In [0]:
# Import Libraries
import pyspark
import delta
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from delta import *
from delta.tables import *

In [0]:
builder = SparkSession.\
          builder.\
          appName("MyApp"). \
          config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension"). \
          config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"). \
          config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0")

spark = configure_spark_with_delta_pip(builder).\
        getOrCreate()

In [0]:
spark.conf.set("fs.azure.account.auth.type.degroup1.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.degroup1.dfs.core.windows.net",  "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.degroup1.dfs.core.windows.net", "43a454c9-f390-4cef-9053-dd6ea96fe7e4")
spark.conf.set("fs.azure.account.oauth2.client.secret.degroup1.dfs.core.windows.net","3e20ee5a-4949-41a8-821d-8d270b11ff65")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.degroup1.dfs.core.windows.net", "https://login.microsoftonline.com/9511cbf1-667c-419c-be5a-7f61d032b106/oauth2/token")

In [0]:
data = spark.range(0, 5)

In [0]:
data.write.format("delta").mode("overwrite").save("abfss://ecommerce@degroup1.dfs.core.windows.net/raw/2023-02-16/*")

In [0]:
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [0]:
df-spark = spark.read.format("delta").load("abfss://ecommerce>@degroup1.dfs.core.windows.net/raw/2023-02-16/*")

In [0]:
df-spark.show()

In [0]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

In [0]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [0]:
from delta.tables import *

In [0]:
deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

In [0]:
# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

In [0]:
# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

In [0]:
# Upsert (merge) new data
newData = spark.range(0, 20)

In [0]:
deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()


In [0]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()

In [0]:
streamingDf = spark.readStream.format("rate").load()

In [0]:
stream = streamingDf.selectExpr("value as id").\
         writeStream.format("delta").\
         option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

In [0]:
stream.stop()

In [0]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()